## Try using aips and difmap coding to simulate Monte Carlo for antenna gain errors

### The operation here:
1. Generate gain facotors for each antenna at each time step(in python)
2. load and correct gain factors (aips, parseltongue)
3. export corrected data to difmap and run modelfit in difmap (python scripting)
4. Save results and analyze in python


In [ ]:
#